In [1]:
%run -i classes_lkl_prof_functions.ipynb

In [2]:
test = lkl_prof('/home2/karwal/lkl_prof/test_lcdm_full_lkl_code/', 'lcdm_cmb_bao_sne_', 'H0')

test.prof_incr = 0.1
test.prof_max = 73.5
test.prof_min = 63.
test.processes = 2

In [3]:
test.mcmc()

Chains converged sufficiently. Current R-1 = 0.030 satisfies R-1 wanted = 0.050. 
Move on to checking minimum.


True

In [4]:
test.global_min()

check_global_min: Found previously run MCMC chains and global minimizer. 
match_param_names: Found existing file with correct name and parameters / parameter sequence. Will append to it. 
/home2/karwal/lkl_prof/test_lcdm_full_lkl_code/lcdm_cmb_bao_sne__p_lkl_profile.txt
match_param_line: Only one entry in file, checking that entry 


{'H0': 67.604448,
 'logA': 3.053544125,
 'n_s': 0.9676225608,
 'omega_b': 0.0224301653,
 'omega_cdm': 0.1195632746,
 'tau_reio': 0.05945180449,
 'A_planck': 0.9999166953,
 'calib_100T': 0.9994798011,
 'calib_217T': 0.9979382208,
 'A_cib_217': 42.01776738,
 'xi_sz_cib': 0.7143866715,
 'A_sz': 6.082866318,
 'ksz_norm': 0.6508100389,
 'gal545_A_100': 9.045358255,
 'gal545_A_143': 10.66944118,
 'gal545_A_143_217': 19.62078032,
 'gal545_A_217': 96.50308403,
 'ps_A_100_100': 244.4176408,
 'ps_A_143_143': 51.51571171,
 'ps_A_143_217': 52.05404186,
 'ps_A_217_217': 125.6662745,
 'galf_TE_A_100': 0.1018342098,
 'galf_TE_A_100_143': 0.1280993332,
 'galf_TE_A_100_217': 0.4970416581,
 'galf_TE_A_143': 0.2207962433,
 'galf_TE_A_143_217': 0.6820259226,
 'galf_TE_A_217': 2.112682567,
 'm_ncdm': 0.06,
 'A_pol': 1.0,
 'calib_100P': 1.021,
 'calib_143P': 0.966,
 'calib_217P': 1.04,
 'cib_index': -1.3,
 'A_sbpx_100_100_TT': 1.0,
 'A_sbpx_143_143_TT': 1.0,
 'A_sbpx_143_217_TT': 1.0,
 'A_sbpx_217_217_TT': 

In [5]:
test.init_lkl_prof()

{'theory': {'classy': {'path': '/home2/karwal/AxiCLASS/',
   'speed': 0.2,
   'stop_at_error': False,
   'extra_args': {'N_ncdm': 1, 'N_ur': 2.0328, 'non linear': 'hmcode'},
   'use_renames': False,
   'renames': {'As': 'A_s',
    'ns': 'n_s',
    'nrun': 'alpha_s',
    'nrunrun': 'beta_s',
    'nt': 'n_t',
    'ntrun': 'alpha_t',
    'rdrag': 'rs_drag',
    'omegak': 'Omega_k',
    'omegal': 'Omega_Lambda',
    'w': 'w0_fld',
    'wa': 'wa_fld',
    'omegabh2': 'omega_b',
    'omegab': 'Omega_b',
    'omegach2': 'omega_cdm',
    'omegac': 'Omega_cdm',
    'omegam': 'Omega_m',
    'omegan': 'Omega_nu',
    'tau': 'tau_reio',
    'zrei': 'z_reio',
    'deltazrei': 'reionization_width',
    'helium_redshift': 'helium_fullreio_redshift',
    'helium_delta_redshift': 'helium_fullreio_width',
    'yhe': 'YHe',
    'yheused': 'YHe'},
   'version': 'v2.9.4',
   'input_params': ['A_s',
    'H0',
    'm_ncdm',
    'n_s',
    'omega_b',
    'omega_cdm',
    'tau_reio'],
   'output_params': ['Ome

In [ ]:
test.run_lkl_prof()

## Analysis functions 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
chis = test.sum_params(['chi2__CMB', 'chi2__SN', 'chi2__BAO'])
lkl_prof_param = test.full_lkl_prof_dict()[test.prof_param]

In [ ]:
plt.plot( lkl_prof_param , chis )
plt.ylabel(r'$\chi^2$')
plt.xlabel(r'$'+lkl_pro_yaml['params'][test.prof_param]['latex']+'$')
plt.show()

In [ ]:
prof_lkl_output = test.full_lkl_prof_dict()

for param in test.param_order:
    plt.plot( prof_lkl_output[test.prof_param], prof_lkl_output[param] )
    try:
        plt.ylabel(r'$'+lkl_pro_yaml['params'][param]['latex']+'$')
    except KeyError:
        plt.ylabel([param])
    plt.xlabel(r'$'+lkl_pro_yaml['params'][test.prof_param]['latex']+'$')
#     plt.ylabel(param)
#     plt.xlabel(test.prof_param)
    plt.show()